# GDAL Test Notebook to check usage of GDAL library

To avoid non compatibility between packages at the installation, install everything in one shot 
conda install -c conda-forge gdal matplotlib scikit-image tqdm tensorflow 

In [1]:
from osgeo import gdal

In [2]:
import matplotlib

In [3]:
import tensorflow

In [7]:
file_path = r"C:\Users\VArri\Documents\Rooftop\dataset\dataset\dataset\train\images\austin1.tif"
raster = gdal.Open(file_path)
type(raster)

osgeo.gdal.Dataset

In [9]:
# Projection
print(raster.GetProjection())

# Dimensions
print(raster.RasterXSize)
print(raster.RasterYSize)

# Number of bands
print(raster.RasterCount)

# Metadata for the raster dataset
print(raster.GetMetadata())

PROJCS["NAD83 / UTM zone 14N",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-99],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","26914"]]
5000
5000
3
{'AREA_OR_POINT': 'Area'}


In [16]:
ulx, xres, xskew, uly, yskew, yres  = raster.GetGeoTransform()
# Xp = padfTransform[0] + P*padfTransform[1] + L*padfTransform[2];
# Yp = padfTransform[3] + P*padfTransform[4] + L*padfTransform[5];
# In a north up image, padfTransform[1] is the pixel width, and padfTransform[5] is the pixel height. 
# The upper left corner of the upper left pixel is at position (padfTransform[0],padfTransform[3]).
print(raster.GetGeoTransform())

(616500.0, 0.29999999999997673, 0.0, 3345000.0, 0.0, -0.30000000000009314)


In [18]:
raster.GetProjection()

'PROJCS["NAD83 / UTM zone 14N",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-99],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","26914"]]'

In [19]:
!gdalinfo $file_path

Driver: GTiff/GeoTIFF
Files: C:\Users\VArri\Documents\Rooftop\dataset\dataset\dataset\train\images\austin1.tif
Size is 5000, 5000
Coordinate System is:
PROJCRS["NAD83 / UTM zone 14N",
    BASEGEOGCRS["NAD83",
        DATUM["North American Datum 1983",
            ELLIPSOID["GRS 1980",6378137,298.257222101,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4269]],
    CONVERSION["UTM zone 14N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-99,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
           

In [20]:
!gdalwarp -tr 0.6 0.6 $file_path cvt.tif

Creating output file that is 2500P x 2500L.
Processing C:\Users\VArri\Documents\Rooftop\dataset\dataset\dataset\train\images\austin1.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


In [23]:
!gdalinfo cvt.tif

Driver: GTiff/GeoTIFF
Files: cvt.tif
Size is 2500, 2500
Coordinate System is:
PROJCRS["NAD83 / UTM zone 14N",
    BASEGEOGCRS["NAD83",
        DATUM["North American Datum 1983",
            ELLIPSOID["GRS 1980",6378137,298.257222101,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4269]],
    CONVERSION["UTM zone 14N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-99,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["metre",1],
            ID["EPSG",8806]],
        PARAMETER["F

In [27]:
ext = GetExtent(raster) 
ext

((616500.0, 3345000.0),
 (617999.9999999999, 3345000.0),
 (617999.9999999999, 3343499.9999999995),
 (616500.0, 3343499.9999999995))

In [28]:
def GetExtent(ds):
    """ Return list of corner coordinates from a gdal Dataset """
    xmin, xpixel, _, ymax, _, ypixel = ds.GetGeoTransform()
    width, height = ds.RasterXSize, ds.RasterYSize
    xmax = xmin + width * xpixel
    ymin = ymax + height * ypixel

    return (xmin, ymax), (xmax, ymax), (xmax, ymin), (xmin, ymin)

# https://gis.stackexchange.com/questions/57834/how-to-get-raster-corner-coordinates-using-python-gdal-bindings